In [ ]:
import sys


path = "/home/someone/stage_jonathan/lightning-sam/lightning-sam/l_sam_env/lib/python3.10/site-packages"

if path not in sys.path:
    sys.path.append(path)

In [ ]:
from config import cfg

In [ ]:
path_to_img = cfg.dataset.img_dir

In [ ]:
import os 
import cv2
import matplotlib.pyplot as plt

In [ ]:
file = os.listdir(path_to_img)[150]
print(file)

In [ ]:
file+"/"+os.listdir(os.path.join(path_to_img,file))[6]

In [ ]:
img = cv2.imread(os.path.join(path_to_img+"/"+file, 
                              os.listdir(os.path.join(path_to_img,
                                                      file))[6]))

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:

box_file = cfg.dataset.box_file
img_dir = cfg.dataset.img_file


with open(box_file,"r") as f:
    temp = f.readlines()

    # format i x1 y1 x2 y2 
    boxes = [list(map(lambda x : int(float(x)), line.split()[1:])) for line in temp] 

with open(img_dir,"r") as f:
    temp = f.readlines()
    # format i folder_name/image_name
    img_names = [os.path.join(path_to_img,line.split()[1]) for line in temp]

In [ ]:
from custom_transform import LocalizedRandomResizedCrop

In [ ]:
idx = [(i,name) for (i,name) in enumerate(img_names) if  file+"/"+os.listdir(os.path.join(path_to_img,file))[6] in name ][0]

In [ ]:
boxes[idx[0]]

In [ ]:
# convert to PIL
from PIL import Image
from matplotlib.patches import Rectangle

img_pil = Image.fromarray(img)

# (x,y) , w, h
fig, ax = plt.subplots()
ax.imshow(img_pil)
x,y,xw,yh = boxes[idx[0]]


rect = Rectangle((x,y),xw,yh, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()

In [ ]:
import torchvision.transforms as T
import random as rd

# T.RandomPerspective(distortion_scale=0.2, p=1, fill = 255*rd.random())
#T.ElasticTransform(alpha=10.0,sigma=2.0)

new_t = T.Compose([
    T.Resize(224),
    T.CenterCrop(224),])

new = new_t(img_pil)

print(new.size)
plt.imshow(new)

plt.axis("off")
plt.show()

In [ ]:
%%timeit 
T.AugMix(severity=2)(img_pil)

In [ ]:
%%timeit
p= 0.5
T.ColorJitter(brightness=p*0.5, contrast=p*0.5, saturation=p*0.5, hue=p*0.1)(img_pil)

In [ ]:
idx

In [ ]:
x,y,w,h = boxes[idx[0]]

cropped = LocalizedRandomResizedCrop(img_pil,*[x,y,w,h],size=300, THR=1, relative_upper_scale=1.0, ratio=(1.0,1.0))
plt.figure(figsize=(10,10))
plt.imshow(cropped)
plt.axis("off")
plt.show()

In [ ]:
from compute_iou import get_miou

import numpy as np


for thr in np.linspace(0,1,11):
    local, rand = get_miou(thr)
    
    plt.hist(local, bins=100, alpha=0.5, label='local')
    plt.hist(rand, bins=100, alpha=0.5, label='rand')
    plt.legend(loc='upper left')
    mean = np.mean(local)
    print(mean)
    plt.title(f"comparison for thr = {thr:.4f}")
    plt.show()
    



import numpy as np
thr = 0.001

local, rand = get_miou(thr)

plt.figure(figsize=(16,10))
plt.hist(local, bins=100, alpha=0.5, label='local')
plt.hist(rand, bins=100, alpha=0.5, label='rand')
plt.legend(loc='upper left')
mean = np.mean(local)
plt.title(f"comparison for thr = {thr}, mean = {mean}")
plt.grid()
plt.show()

len(np.where(np.array(local)<0.9
            )[0])

In [ ]:
from dataset import load_cub_datasets
cfg.use_box = True
cfg.THR = 1
cfg.augment = True
cfg.augment_p = 1
cfg.use_augment_mix = False
train_local, val_local = load_cub_datasets(cfg)


cfg.THR = 1
cfg.use_box = True
cfg.augment = False
train_rando, val_rando = load_cub_datasets(cfg)

In [ ]:
for (imgs1, labels1), (imgs2, labels2) in zip(train_local,train_rando):
    for img1, img2 in zip(imgs1,imgs2):
        img1 = img1.permute((1,2,0))
        img1 -= img1.min()
        img1/=img1.max()
        
        img2 = img2.permute((1,2,0))
        img2 -= img2.min()
        img2/=img2.max()
        
        plt.subplot(1,2,1)
        plt.imshow(img1)
        plt.subplot(1,2,2)
        plt.imshow(img2)
        plt.show()

        
    break

In [ ]:
from dataset import load_box_dataset
from config import cfg
import os

seg = os.path.join(
    os.path.dirname(cfg.dataset.img_dir),
    "masks_0"
)

assert os.path.exists(seg)

seg_file = os.path.join(seg,
                       "masks_info.json")
train, val = load_box_dataset(cfg,seg_file)

In [ ]:
for img1s, idxs, labels in train:
    for img1, idx, label in zip(img1s, idxs, labels):
        print(idx,label)
        img1 = img1.permute((1,2,0))
        img1 -= img1.min()
        img1/=img1.max()
        plt.imshow(img1)
        plt.show()

        
    break
        

In [ ]:
import numpy as np
maiou = np.linspace(0.4,1)
THR = (2*(maiou-0.5))**(1/0.4)
plt.grid()
plt.plot(maiou,THR)

In [ ]:
(112*60)/2.5

In [ ]:
1/0

In [ ]:
X = []
Y = []
for e in np.concatenate([np.linspace(0,0.2,10),np.linspace(0.21,0.7,10), np.linspace(0.8,1,3)]):
    a, _ = get_miou(THR=e)
    X.append(np.mean(a))
    Y.append(e)
    
    

In [ ]:
vec = np.mean(np.array(X),axis=1)

In [ ]:
len(vec) == len(Y)

In [ ]:
plt.figure(figsize=(14,14))
plt.plot(Y,vec)
plt.xticks(np.linspace(0,1, 21))
plt.yticks(np.linspace(0.5,1, 21))
plt.grid()
plt.title("IOU assym. moyenne en fonction de THR")
plt.show()

In [ ]:
L = []
for dir in os.listdir(cfg.dataset.img_dir):
    L.append(len(os.listdir(os.path.join(cfg.dataset.img_dir,dir))))

print(np.mean(L))
print(np.std(L))
print(np.min(L))
print(np.max(L))

In [ ]:
import json

for name in ["masks_0","masks_1","masks_2", "masks_3"]:

    root_dir = os.path.dirname(cfg.dataset.img_dir)

    masks_dir = os.path.join(root_dir, name)

    with open(os.path.join(masks_dir, "masks_info.json"), "r") as f:
        d = json.load(f)

    print(name)
    print(d["setup"])

In [ ]:
with open(cfg.dataset.img_file,"r") as f:
        temp = f.readlines()
        # format i path
        img_names = {int(line.split()[0]):line.split()[1] for line in temp} # index : path


with open(cfg.dataset.label_file,"r") as f:
            temp = f.readlines()
            # format i class_number
            labels = {int(line.split()[0]) : int(line.split()[1])-1 for line in temp}

with open(cfg.dataset.box_file,"r") as f:
    temp = f.readlines()
    # format i x y w h
    boxes = {int(line.split()[0]):list(map(lambda x : int(float(x)), line.split()[1:])) for line in temp}

In [ ]:

import numpy as np
from tqdm import tqdm

def compute_iou(bbox,gt,):

    """
    Args:
        y, x, h, w
    """

    x1, y1, w1, h1 = bbox
    x3, y3, w3, h3 = gt

    x2 = x1 + w1
    y2 = y1 + h1

    x4 = x3 + w3
    y4 = y3 + h3

    #  x1,y1,x2,y2,x3,y3,x4,y4

    inter_width = min(x2, x4) - max(x1, x3)
    inter_height = min(y2, y4) - max(y1, y3)

    if inter_width <= 0 or inter_height <= 0:
        return 0
    
    areaIntersection = inter_width * inter_height 

    areaUnion = w1 * h1 + w3 * h3 - areaIntersection

    return areaIntersection / (areaUnion+1e-6)

L_max_iou = []

for idx in tqdm(range(1,1+len(img_names))):

    name = img_names[idx]
    box = boxes[idx]

    L_max_iou.append(
        max(
            [compute_iou(box, d[name][i]["bbox"]) for i in range(len(d[name]))]
        )
    )

print(np.mean(L_max_iou))
    



In [ ]:
d[img_names[1]][4]

In [ ]:
import torchvision
import torch.nn as nn

model = torchvision.models.resnet50(weights=None)

model.fc = nn.Linear(model.fc.in_features,cfg.model.n_classes)

model.load_state_dict(torch.load(cfg.model.resumed_model))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device=device)

In [ ]:
import random as rd

X = []

Y = []

for _ in range(50):

    dummy_maiou = rd.uniform(0.5,1.05)


    

    if 0.5 <= dummy_maiou <= 1.0:
                thr = (2*(dummy_maiou-0.5))**(1/0.4)

                maiou = dummy_maiou

    else: # 1 < dummy_maiou 
        thr = -1 # will not use box

        maiou = 1.4 + (dummy_maiou-1.0)

    X.append(maiou)

    if maiou > 1:
        y = 0.76 + rd.uniform(-0.005,0.005)
    
    else:
        y = 0.76 + rd.uniform(-0.005,0.005)

    Y.append(y)

plt.scatter(X,Y)
plt.show()

In [ ]:
import torch


from PIL import Image
from torchvision import transforms

import os

from config import cfg

torch.cuda.empty_cache()
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)


"""transforms.Compose([
transforms.RandomRotation(degrees=(1,1)),
transforms.Resize((new_w, new_h), antialias=True),
]),
transforms.Compose([
transforms.RandomRotation(degrees=(2,2)),
transforms.Resize((new_w, new_h), antialias=True),
]),
transforms.Compose([
transforms.RandomRotation(degrees=(3,3)),
transforms.Resize((new_w, new_h), antialias=True),
]),
transforms.Compose([
transforms.RandomRotation(degrees=(-1,-1)),
transforms.Resize((new_w, new_h), antialias=True),
]),
transforms.Compose([
transforms.RandomRotation(degrees=(-2,-2)),
transforms.Resize((new_w, new_h), antialias=True),
]),
transforms.Compose([
transforms.RandomRotation(degrees=(-3,-3)),
transforms.Resize((new_w, new_h), antialias=True),
])"""

In [ ]:
with open(cfg.dataset.img_file,"r") as f:
        temp = f.readlines()
        # format i path
        img_names = {int(line.split()[0]):line.split()[1] for line in temp} # index : path


with open(cfg.dataset.label_file,"r") as f:
            temp = f.readlines()
            # format i class_number
            labels = {int(line.split()[0]) : int(line.split()[1])-1 for line in temp}

with open(cfg.dataset.box_file,"r") as f:
    temp = f.readlines()
    # format i x y w h
    boxes = {int(line.split()[0]):list(map(lambda x : int(float(x)), line.split()[1:])) for line in temp}

In [ ]:


#root / folder / img_name

import matplotlib.pyplot as plt
import cv2

root = cfg.dataset.img_dir
folder = os.listdir(root)[1]
img_name = os.listdir(os.path.join(root, folder))[19]

path_to_img = os.path.join(root, folder, img_name)

path_to_img = "/mnt/data/CUB_200_2011/images/108.White_necked_Raven/White_Necked_Raven_0070_102645.jpg"
img = cv2.imread(path_to_img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.show()

def preprocess_img(path_to_img, resize=False):
    """img = cv2.imread(path_to_img)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = Image.fromarray(img)"""

    img = Image.open(path_to_img)
    if img.mode != "RGB":
        img = img.convert("RGB")
    
    if resize:img = transforms.Resize(500, antialias=True)(img)
    img = transforms.RandomAutocontrast(p=1)(img)

    h, w = img.size
    new_w = w - (w % 14)
    new_h = h - (h % 14)


    img = transforms.ToTensor()(img)

    img = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])(img) 

    img = transforms.Resize((new_w, new_h), antialias=True)(img)

    img = img.unsqueeze(0).to(device)

    return img

img = preprocess_img(path_to_img)



In [ ]:


def get_last_attention(model, imgs,):

    new_w = img.shape[-2]
    new_h = img.shape[-1]

    w_featmap = new_w // 14
    h_featmap = new_h // 14

    with torch.no_grad():
        output = model.get_intermediate_layers(x=imgs,
                                            reshape=True,
                                            n = 2,
                                            return_class_token=True,
                                            )

        # output is a list of tuples (maps, class_token), the length of the list is the number of layers considered


        maps = output[0][0] 

        B, C = output[0][1].shape

        # reshape maps to be (B, N, C) where N is the number of patches

        maps = maps.reshape((B,maps.shape[1],-1)).permute(0,2,1)


        class_token = output[0][1].reshape((B,-1,1)).permute(0,2,1)


        maps = torch.cat((class_token, maps), dim=1)

        # get the last attention block (only qkv)with 
    
        qkv = model.blocks[-1].attn.qkv


        B, N, C = maps.shape

        qkv_out = qkv(maps).reshape(B, N, 3, model.num_heads, C // model.num_heads).permute(2, 0, 3, 1, 4) # (3, B, num_heads, N, C//num_heads)


        head_dim = C // model.num_heads
        scale = head_dim**-0.5

        q, k = qkv_out[0] * scale, qkv_out[1]

        attn = q @ k.transpose(-2, -1) # (B, num_heads, N, N)
        
        nh = model.num_heads
        assert B == 1, "B must be 1"
        attn = attn[:, :, 0, 1:].reshape(nh, w_featmap, h_featmap)
        
        return attn



In [ ]:
%%timeit
attentions = get_last_attention(model, img,)

In [ ]:
def LocalizedRandomResizedCrop(
                image, 
                xo, yo, Wo, Ho,
                THR: float = 1.0,
                scale: tuple = (0.08, 1.0),
                ratio: tuple = (3. / 4., 4. / 3.),
                relative_upper_scale = None, # set to 1 when cropping bbox with non square aspect ratio (very stretched)
                ):
    
        """
        Args :
            image (PIL Image): Image to be cropped.
            bbox (tuple): Bounding box coordinates (x, y, w, h)
            size (sequence or int): Desired output size of the crop. If size is an
                int instead of sequence like (h, w), a square crop (size, size) is
                made.

                should be a multiple of patch_size
            alpha (float): 0: centers are equal, 1 : centers are shifted
            scale (tuple): Range of the random size of the cropped image compared to
                the original image size.
            ratio (tuple): Range of aspect ratio of the origin aspect ratio cropped
                image compared to the original image ratio. Default value is
                (3. / 4., 4. / 3.).

        Returns:
            PIL Image: Cropped image.
    
        """


        area_image = image.size[0] * image.size[1]

        Ao = max(Wo, Ho)**2

        if relative_upper_scale is None:
            scale = (max(scale[0], (THR*Ao)/area_image),
                    #min(scale[1], (1/(THR + 1e-8))*Ao/area_image))
                    scale[1])
        else:
             scale = (max(scale[0], (THR*Ao)/area_image),
                    min(scale[1], relative_upper_scale)*Ao/area_image)
        
        effective_scale = rd.uniform(*scale)
        log_ratio = tuple(np.log(r) for r in ratio)

        effective_ratio = np.exp(rd.uniform(*log_ratio))

        side = area_image**0.5  

        crop_side = side * effective_scale**0.5

        if effective_ratio > 1:
            Wc = effective_ratio * crop_side
            Hc = crop_side
        else:
            Wc = crop_side
            Hc = crop_side / effective_ratio

        # prevent overflow
        Hc = min(Hc, float(image.size[1]))
        Wc = min(Wc, float(image.size[0]))

        alpha = 1 - THR**0.5

        range_x = alpha * (Wc + Wo)/2
        range_y = alpha * (Hc + Ho)/2

        effective_x = rd.uniform(-range_x, range_x)
        effective_y = rd.uniform(-range_y, range_y)

        center_x = xo + Wo/2 + effective_x
        center_y = yo + Ho/2 + effective_y
        crop_bbox = [

            max(0, center_y - Hc/2),
            max(0, center_x - Wc/2),
            
            Hc,
            Wc,
            
        ]

        if crop_bbox[0] + crop_bbox[2] > image.size[1]:
            crop_bbox[2] = image.size[1] - crop_bbox[0]
        if crop_bbox[1] + crop_bbox[3] > image.size[0]:
            crop_bbox[3] = image.size[0] - crop_bbox[1]

        crop_bbox = [abs(int(x)) for x in crop_bbox] # avoid crash with negative values

        return crop_bbox

In [ ]:
def iou(gt,bbox, mode="std"):
    """
    gt and bbox :  x,y, w, h
    """
    x1, y1, w1, h1 = gt
    x2, y2, w2, h2 = bbox

    area1 = w1*h1
    area2 = w2*h2

    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1+w1, x2+w2)
    yB = min(y1+h1, y2+h2)
    
    interArea = max(0, xB-xA) * max(0, yB-yA)
    if mode == "std":
        unionArea = area1 + area2 - interArea
    elif mode == "as-gt":
        unionArea = area1
    elif mode == "as-bbox":
        unionArea = area2
    else:
        raise ValueError("mode must be one of 'std', 'as-gt', 'as-bbox'")

    eps = 1e-6

    iou = interArea / max(unionArea,eps)

    return iou

In [ ]:
cfg.dataset.split_file = "/mnt/data/CUB_200_2011/train_test_split.txt"

with open(cfg.dataset.split_file,"r") as f:
    temp = f.readlines()
    # format i class_number
    split = {int(line.split()[0]) : int(line.split()[1]) for line in temp}


In [ ]:
from copy import deepcopy

cls_token = deepcopy(
    model.cls_token
)

print(cls_token.shape)

In [ ]:
import torch.nn as nn
model.cls_token = nn.Parameter(
    torch.load("/home/someone/stage_jonathan/beyond_sota_w_sam/models/Dino_train_ac87.369_2023-08-24_01:27:15.pt")["cls_token"]
)

In [ ]:
for i,name in enumerate(os.listdir(root)):
    if "humming" in name.lower():
        print(i,name)

In [ ]:
root_imagenet = "/mnt/data/fred/datasets/miniimagenetimages/images/"

In [ ]:
import numpy as np
from scipy.signal import convolve2d
from matplotlib.patches import Rectangle
from matplotlib.patches import Ellipse
from tqdm import tqdm

import random as rd

import cv2

L_iou = []

d_box = {}
#rd.seed(42)
#img_idxs = rd.sample(list(boxes.keys()),1000)

for img_idx in range(1):

    """folder = os.listdir(root)[164]#[rd.randint(0, len(os.listdir(root)) - 1)]
    img_name = os.listdir(os.path.join(root, folder))[rd.randint(0, len(os.listdir(os.path.join(root, folder))) - 1)]"""


    img_name = os.path.join(
        root_imagenet,
        os.listdir(root_imagenet)[rd.randint(0, len(os.listdir(root_imagenet)) - 1)]
    )

    #path_to_img = os.path.join(root, folder, img_name)

    #img_idx = img_idxs[i]

    path_to_img = os.path.join(root, img_name)

    #path_to_img = "/home/someone/Pictures/Screenshots/Screenshot from 2023-08-17 09-19-08.png"

    path_to_img = "/home/someone/Pictures/Screenshots/Screenshot from 2023-08-29 09-33-59.png"

    try:
        img = preprocess_img(path_to_img, resize=True)
    except Exception as e:
        print("error with", path_to_img)
        print(e)
        continue

    attentions = get_last_attention(model, img,).cpu().numpy()



    # remove the n highest values
    for attention in attentions:
        for  _ in range(1):
            idx = np.unravel_index(np.argmax(attention), attention.shape)
            attention[idx] = np.mean(attention)

    plt.figure(figsize=(10,6), dpi=200)

    alpha = 4 # temperature

    softmax = lambda x: np.exp(alpha*x)/np.sum(np.exp(alpha*x))

    plt.subplot(1, 3, 1)
    plt.title("Original",size=6)
    to_display = img[0].permute(1,2,0).cpu().numpy()
    to_display = (to_display - to_display.min())/(to_display.max() - to_display.min())
    plt.imshow(to_display)
    
    plt.axis("off")

    #gt = boxes[img_idx]
    """plt.gca().add_patch(Rectangle((gt[0],gt[1]), gt[2], gt[3],
                        edgecolor='blue',
                        facecolor='none',
                        lw=1))

    plt.subplot(1, 3, 1)"""

    conved  = convolve2d(
                np.mean(attentions,axis=0),
                np.ones(
                    tuple(max(3,int(0.2*side)) for side in attentions.shape[1:])
                ),
                mode="same",
                boundary="symm"
            )
    conved = cv2.resize(
                conved,
                (to_display.shape)[:-1:][::-1],
                interpolation=cv2.INTER_CUBIC
    )
    conved = (conved - conved.min())/(conved.max() - conved.min())

    plt.subplot(1, 3, 2)

    plt.title("Softmax + box estimate",size=6)

    softmaxed = softmax(conved)
    softmaxed = (softmaxed - softmaxed.min())/(softmaxed.max() - softmaxed.min())

    plt.imshow(softmaxed,cmap="coolwarm")

    threshed = cv2.threshold(
                (255*softmaxed).astype(np.uint8),
                75, 255,
                cv2.THRESH_BINARY #+ cv2.THRESH_OTSU
                )[1]/255

    center_y, center_x = np.mean(np.argwhere(threshed),axis=0)

    h = np.sum(threshed.max(axis=1),axis=0)
    w = np.sum(threshed.max(axis=0),axis=0)

    plt.scatter(center_x,center_y,marker="x",color="red")

    x = center_x - w/2
    y = center_y - h/2

    """L_iou.append(iou(gt,
                     [x,y,w,h],
                     mode="std"))"""
    
    d_box[img_idx] = list(map(int,[x,y,w,h]))
    
    #print(f"{img_idx:4d}/{len(list(boxes.keys()))} : {float(np.mean(L_iou)):.3f}", end="\r")
    #print(L_iou[-1])

    plt.gca().add_patch(Rectangle((x,y), w, h,
                        edgecolor='red',
                        facecolor='none',
                        lw=1))
    

    """AssertionError"""


    
    
    plt.axis("off")
    

    plt.subplot(1, 3, 1)
    



    L = []
    for _ in range(10):
        bbox = LocalizedRandomResizedCrop(
                Image.fromarray(softmaxed*255).convert("L"),
                x, y, w, h,
                THR=0.7,
                scale=(0.08, 0.5),
                ratio=(3. / 4., 4. / 3.),
                relative_upper_scale = None, # set to 1 when cropping bbox with non square aspect ratio (very stretched)
                )

        """plt.gca().add_patch(Ellipse((bbox[1]+bbox[3]/2,bbox[0]+bbox[2]/2), bbox[3], bbox[2],
                        edgecolor='green',
                        facecolor='none',
                        lw=1))"""
        
        plt.gca().add_patch(Rectangle((bbox[1],bbox[0]), bbox[3], bbox[2],
                        edgecolor='green',
                        facecolor="none",
    
                        lw=1))              
    


    plt.axis("off")


    plt.subplot(1, 3, 3)
    plt.title("Otsu thr",size=6)


    plt.imshow(threshed,cmap="coolwarm")
    plt.axis("off")
    plt.show()




In [ ]:
img.shape

In [ ]:
def get_self_attention(model,imgs):

    """
    Only works for ViT models (especially DINO, DINOv2)
    
    Args:
        model: A ViT model
        imgs: A tensor of shape (B, 3, H, W) B = 1 
    Returns:
        A tensor of shape (num_heads, H_new, W_new) where H_new and W_new are the height and width of the attention map (size of the img divided by the patch size), CUDA tensor
    """

    B, C, H, W = imgs.shape

    assert len(imgs.shape) == 4, "imgs must be a 4D tensor"
    assert imgs.shape[1] == 3, "imgs must have 3 channels"
    assert imgs.shape[2] % model.patch_size == 0 and imgs.shape[3] % model.patch_size == 0, "sides must be divisible by 14"
    assert B == 1, "B must be 1"

    w_featmap = W // 14
    h_featmap = H // 14

    with torch.no_grad():
        output = model.get_intermediate_layers(x=imgs,
                                            reshape=True,
                                            n = 2,
                                            return_class_token=True,
                                            )

        # output is a list of tuples (maps, class_token), the length of the list is the number of layers considered


        maps = output[0][0] 

        B, C = output[0][1].shape

        # reshape maps to be (B, N, C) where N is the number of patches

        maps = maps.reshape((B,maps.shape[1],-1)).permute(0,2,1)


        class_token = output[0][1].reshape((B,-1,1)).permute(0,2,1)


        maps = torch.cat((class_token, maps), dim=1)

        # get the last attention block (only qkv) with 
    
        qkv = model.blocks[-1].attn.qkv


        B, N, C = maps.shape

        qkv_out = qkv(maps).reshape(B, N, 3, model.num_heads, C // model.num_heads).permute(2, 0, 3, 1, 4) # (3, B, num_heads, N, C//num_heads)


        head_dim = C // model.num_heads
        scale = head_dim**-0.5

        q, k = qkv_out[0] * scale, qkv_out[1]

        attn = q @ k.transpose(-2, -1) # (B, nh, N, N)
        
        nh = model.num_heads


        #attn = attn[:, :, 0, 1:].reshape(nh, h_featmap, w_featmap)
        
        return attn, output[0][0] 



In [ ]:
a, maps = get_self_attention(model, img,)
a = a.squeeze(0).mean(axis=0)[:-1,:-1]
for i in range(a.shape[0]):
    a[i,i] = 0




In [ ]:
mean = torch.mean(a,axis=1)

seeds = mean.cpu().numpy().reshape(35,47)


plt.imshow(seeds, cmap="coolwarm")

plt.colorbar()



In [ ]:
sorted_seeds = np.argsort(seeds.flatten())[:100]

print(np.unravel_index(sorted_seeds[0], seeds.shape))

In [ ]:
vec = np.zeros(seeds.flatten().shape)

first_seed = sorted_seeds[0]

for seed in sorted_seeds[1:]:
    idx =  seed

    if maps[idx].cpu().numpy().T @ maps[first_seed].cpu().numpy() > 0:
        vec[idx] = 1


vec2 = np.zeros(vec.shape)

for seed in vec:
    seed = int(seed)
    if vec[seed] == 1:

        value = maps[seed].cpu().numpy().T @ np.sum([maps[i].cpu().numpy() for i in range(vec.shape[0]) if vec[i] == 1 and i != seed],axis=0) > 0

        vec2[seed] = value



In [ ]:
plt.imshow(vec2.reshape(35,47), cmap="coolwarm")

In [ ]:
import torch
attn = get_last_attention(model, img,)

# remove max per head

for att in attn:
    att[att == att.max()] = att.mean()
attn = torch.mean(attn,dim=0)


In [ ]:
import torchvision.transforms.functional as F

conv = torch.nn.Conv2d(1,1,3, 
                       padding="same", 
                       padding_mode="reflect",
                       bias=False)
conv.weight = torch.nn.Parameter(torch.ones(3,3).unsqueeze(0).unsqueeze(0))
conv.to("cuda")


conved = conv(attn.unsqueeze(0).unsqueeze(0)).squeeze(0).squeeze(0)

conved = F.resize(conved.unsqueeze(0), attn.shape, antialias=True).squeeze(0)

print(conved.shape)

softmax = torch.nn.Softmax(dim=0)


conved = (conved - conved.min())/(conved.max() - conved.min())
softmaxed = softmax(100*conved.flatten())

softmaxed = softmaxed.reshape(conved.shape)

print(softmaxed.shape)

plt.imshow(softmaxed.detach().cpu().numpy(), cmap= "coolwarm")
plt.colorbar()
plt.show()
plt.imshow(conved.detach().cpu().numpy(), cmap= "coolwarm")

plt.colorbar()
plt.show()

In [ ]:
density = softmaxed.flatten()

cum_density = torch.cumsum(density,dim=0)

cum_density = cum_density/cum_density[-1]

plt.imshow(softmaxed.detach().cpu().numpy(), )

for _ in range(100):
    
    u = torch.rand(1).to("cuda")

    idx = torch.searchsorted(torch.tensor(cum_density), u).item()

    idx = np.unravel_index(idx, softmaxed.shape)

    plt.scatter(idx[1],idx[0],color="red")
plt.axis("off")
plt.show()

In [ ]:
attentions = get_last_attention(model, img,).cpu().numpy()

temperature = 0.1

softmax = lambda x: np.exp(x/temperature)/np.sum(np.exp(x/temperature))


for attention in attentions:
        for  _ in range(0):
            idx = np.unravel_index(np.argmax(attention), attention.shape)
            attention[idx] = np.mean(attention)

conved  = convolve2d(
                np.mean(attentions,axis=0),
                np.ones(
                    tuple(max(3,0) for side in attentions.shape[1:])
                ),
                mode="same",
                boundary="symm"
            )



conved = (conved - conved.min())/(conved.max() - conved.min())

softmaxed = softmax(conved)

plt.imshow(
      softmaxed,
)

distribution = np.cumsum(softmaxed.flatten())



for _ in range(100):
    u = rd.random()

    idx = np.argwhere(distribution > u)[0][0]

    point = np.unravel_index(idx, softmaxed.shape)
    plt.scatter(point[1],point[0],color="red", alpha=1)
plt.colorbar()
plt.show()



In [ ]:
import numpy as np
import random as rd
from copy import deepcopy
def crop_from_self_att(map, 
                       #img_size,
                       temperature,
                       n = 1,
                       scale = (0.08, 1.0),
                       ratio = (3. / 4., 1.0),
                       ):
    """
    Args:
        map : resulting distribution after convolution
        img_size : size of the original image
        temperature : temperature used for softmax
        n : number of crops to generate
        scale : scale used for cropping
        ratio : ratio used for cropping
    """
    softmax = lambda x: np.exp(x/temperature)/np.sum(np.exp(x/temperature))
    softmaxed = softmax(map)

    H, W = softmaxed.shape
    area = min(H,W)**2

    print("og",softmaxed.shape)

    crops = []

    for _ in range(n):

        eff_ratio = np.exp(rd.uniform(*np.log(ratio)))
        eff_scale = rd.uniform(*scale)

        w = max(4,int(np.sqrt(area*eff_scale*eff_ratio))-2)
        h = max(4,int(np.sqrt(area*eff_scale/eff_ratio))-2)

        # restrict the space to the center of the image

        #print("size",w,h)

        temp = deepcopy(softmaxed[
            w//2:2-w//2,
            h//2:2-h//2,
        ])

        # normalize
        temp/=np.sum(temp)

        if any(size<=0 for size in temp.shape):
            print("error",h,w,eff_ratio,eff_scale,temp.shape)
        # get the cumulative distribution
        distribution = np.cumsum(temp.flatten())

        #print(distribution[-1])

        # get a random number
        u = rd.random()
        idx = np.argwhere(distribution > u)[0][0]

        point = np.unravel_index(idx, temp.shape)

        # get the coordinates of the crop in the original image
        x = point[0]
        y = point[1] 

        
        crops.append(
            [x,y,w,h]
        )

        # get the coordinates of the crop in the original image
        """x_og = x * img_size[0] / W
        y_og = y * img_size[1] / H

        w_og = w * img_size[0] / W
        h_og = h * img_size[1] / H



        crops.append(
            [x_og,y_og,w_og,h_og]
        )"""

    plt.imshow(softmaxed)
    for x,y,w,h in crops:


        """plt.gca().add_patch(
            Rectangle((x,y),w,h,
                    edgecolor='green',
                    facecolor='none',
                    lw=1)
                      )
    """
        plt.scatter(
            y+h//2,
            x+w//2,
            
        )

    plt.show()


    



crop_from_self_att(map=conved,temperature=0.1,n=100)



In [ ]:
import torch
from torchvision.transforms import functional as F

from typing import Optional, Union, Sequence, List, Tuple
import warnings
import math


class DiNORandomResizedCrop(torch.nn.Module):
      
    """
    Similar to the random resized crop of torchvision, but the distribution 
    of the crop's center is DiNOv2's self attention distribution.

    Should be placed after the Normalize and ToTensor transforms, so that the image is suitable for DiNOv2 (there are no preprocessing transforms)
    """

    def __init__(
    self,
    size,
    dino_type: str,
    scale=(0.08, 1.0),
    ratio=(3.0 / 4.0, 4.0 / 3.0),
    temperature: float = 10,
    kernel_size: int = 3,
    interpolation=F.InterpolationMode.BILINEAR,
    antialias: Optional[Union[str, bool]] = "warn",
    ):
        super().__init__()

        self.size = size

        if not isinstance(scale, Sequence):
            raise TypeError("Scale should be a sequence")
        if not isinstance(ratio, Sequence):
            raise TypeError("Ratio should be a sequence")
        if (scale[0] > scale[1]) or (ratio[0] > ratio[1]):
            warnings.warn("Scale and ratio should be of kind (min, max)")

        self.interpolation = interpolation
        self.antialias = antialias
        self.scale = scale
        self.ratio = ratio


        ############################## Load DiNOv2 model ##############################################
        dino_types = ['dinov2_vits14','dinov2_vitb14','dinov2_vitl14','dinov2_vitg14']
        assert dino_type in dino_types, f"dino_type should be one of {dino_types}, got {dino_type}"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"Loading DiNOv2 model {dino_type} on device {self.device}")
        self.model = torch.hub.load('facebookresearch/dinov2', dino_type, pretrained=True).to(self.device)

        for param in self.model.parameters():
            param.requires_grad = False
        
        self.model.eval()

        self.density_generator = DensityGenerator(temperature=temperature,
                                                  kernel_size=kernel_size,
                                                  ).to(self.device)

        ##############################################################################################

    def get_map(self, img: torch.Tensor) -> torch.Tensor:
        _, height, width = F.get_dimensions(img)
        img = F.resize(img, (height - (height % 14), width - (width % 14)), interpolation=F.InterpolationMode.BICUBIC, antialias=self.antialias)
        _, height, width = F.get_dimensions(img)

        img = img.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        attention_map = get_self_attention(self.model, img) # (num_heads, H//14, W//14)
        density_map = self.density_generator(attention_map).cpu()
        density_map = F.resize(density_map.unsqueeze(0), (height, width), interpolation=F.InterpolationMode.BICUBIC, antialias=self.antialias).squeeze() # (H, W)

        density_map = density_map / torch.sum(density_map)

        return density_map


    def get_params(self, img: torch.Tensor, scale: List[float], ratio: List[float]) -> Tuple[int, int, int, int]:
        """Get parameters for ``crop`` for a random sized crop.

        Args:
            img (PIL Image or Tensor): Input image.
            scale (list): range of scale of the origin size cropped
            ratio (list): range of aspect ratio of the origin aspect ratio cropped

        Returns:
            tuple: params (i, j, h, w) to be passed to ``crop`` for a random
            sized crop.
        """

        ########  Resize for DiNOv2 (so that the image size is divisible by 14)  ######################


        _, height, width = F.get_dimensions(img)

        area = height * width

        # Get distribution of the attention map

        img = img.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        attention_map = get_self_attention(self.model, img) # (num_heads, H//14, W//14)


        # Generate density map

        density_map = self.density_generator(attention_map) # (H//14, W//14)

        density_map = F.resize(density_map.unsqueeze(0), (height, width), interpolation=F.InterpolationMode.BICUBIC, antialias=self.antialias).squeeze() # (H, W)

        log_ratio = torch.log(torch.tensor(ratio))
        for _ in range(10):
            target_area = area * torch.empty(1).uniform_(scale[0], scale[1]).item()
            aspect_ratio = torch.exp(torch.empty(1).uniform_(log_ratio[0], log_ratio[1])).item()

            w = int(round(math.sqrt(target_area * aspect_ratio)))
            h = int(round(math.sqrt(target_area / aspect_ratio)))


            if 0 < w <= width and 0 < h <= height:

                """i = torch.randint(0, height - h + 1, size=(1,)).item()
                j = torch.randint(0, width - w + 1, size=(1,)).item()"""

                x,y = generate_point(density_map, (h,w))

                i = y - h // 2
                j = x - w // 2
            
                return i, j, h, w


        # Fallback to central crop
        in_ratio = float(width) / float(height)
        if in_ratio < min(ratio):
            w = width
            h = int(round(w / min(ratio)))
        elif in_ratio > max(ratio):
            h = height
            w = int(round(h * max(ratio)))
        else:  # whole image
            w = width
            h = height
        i = (height - h) // 2
        j = (width - w) // 2
        return i, j, h, w

    def forward(self, img):
        """
        Args:
            img (PIL Image or Tensor): Image to be cropped and resized.

        Returns:
            PIL Image or Tensor: Randomly cropped and resized image.
        """
        i, j, h, w = self.get_params(img, self.scale, self.ratio)
        return F.resized_crop(img, i, j, h, w, self.size, self.interpolation, antialias=self.antialias)

    def __repr__(self) -> str:
        interpolate_str = self.interpolation.value
        format_string = self.__class__.__name__ + f"(size={self.size}"
        format_string += f", scale={tuple(round(s, 4) for s in self.scale)}"
        format_string += f", ratio={tuple(round(r, 4) for r in self.ratio)}"
        format_string += f", interpolation={interpolate_str}"
        format_string += f", antialias={self.antialias})"
        return format_string


def get_self_attention(model,imgs):

    """
    Extract the self attention map from a ViT model (for now, only spports DINOv2)
    
    Args:
        model: A ViT model
        imgs: A tensor of shape (B, 3, H, W) B = 1 
    Returns:
        A tensor of shape (num_heads, H_new, W_new) where H_new and W_new are the height and width of the attention map (size of the img divided by the patch size), CUDA tensor
    """

    if len(imgs.shape) == 3:
        imgs = imgs.unsqueeze(0)
    elif len(imgs.shape) == 4:
        pass
    else:
        raise ValueError("imgs must be a 3D or 4D tensor")

    B, C, H, W = imgs.shape

    imgs = F.resize(imgs, (H - (H % 14), W - (W % 14)), interpolation=F.InterpolationMode.BICUBIC, antialias=True)
    
    assert imgs.shape[1] == 3, "imgs must have 3 channels"
    #assert imgs.shape[2] % model.patch_size == 0 and imgs.shape[3] % model.patch_size == 0, "sides must be divisible by 14"
    assert B == 1, "B must be 1"



    w_featmap = W // 14
    h_featmap = H // 14

    with torch.no_grad():
        output = model.get_intermediate_layers(x=imgs,
                                            reshape=True,
                                            n = 2,
                                            return_class_token=True,
                                            )

        # output is a list of tuples (maps, class_token), the length of the list is the number of layers considered


        maps = output[0][0] 

        B, C = output[0][1].shape

        # reshape maps to be (B, N, C) where N is the number of patches

        maps = maps.reshape((B,maps.shape[1],-1)).permute(0,2,1)


        class_token = output[0][1].reshape((B,-1,1)).permute(0,2,1)


        maps = torch.cat((class_token, maps), dim=1)

        # get the last attention block (only qkv) with 
    
        qkv = model.blocks[-1].attn.qkv


        B, N, C = maps.shape

        qkv_out = qkv(maps).reshape(B, N, 3, model.num_heads, C // model.num_heads).permute(2, 0, 3, 1, 4) # (3, B, num_heads, N, C//num_heads)


        head_dim = C // model.num_heads
        scale = head_dim**-0.5

        q, k = qkv_out[0] * scale, qkv_out[1]

        attn = q @ k.transpose(-2, -1) # (B, nh, N, N)
        
        nh = model.num_heads
        attn = attn[:, :, 0, 1:].reshape(nh, h_featmap, w_featmap)
        
        return attn


class DensityGenerator(torch.nn.Module):
    """
    Postprocess the attention maps to generate a density map : 
    - Average over the heads
    - Outlier removal (remove the max and replace it by the mean)
    - Apply a gaussian filter
    - Aplly a min max normalization
    - Apply a softmax


    Args:
        kernel_size: The size of the gaussian kernel (Note that all images should be the same size, and are square)
        temperature: The temperature used to apply the softmax
    """

    def __init__(self, kernel_size: int = 3, temperature: float = 0.1):
        super().__init__()
        self.kernel_size = kernel_size
        self.temperature = temperature

        self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=kernel_size, padding="same", bias=False) # padding to keep the same size
        self.conv.weight.data = torch.ones(1, 1, kernel_size, kernel_size)

        self.softmax = torch.nn.Softmax(dim=0)

    def forward(self, maps: torch.Tensor) -> torch.Tensor:
        """
        Args:
            maps: A tensor of shape (num_heads, H, W)
        Returns:
            A tensor of shape (H, W)
        """

        maps = torch.mean(maps, dim=0) # average over the heads # (H, W)

        maps = self.conv(maps.unsqueeze(0)) # gaussian filter # (1, H, W)

        # positive values

        maps = (maps - torch.min(maps))
        
        temp = maps.clone()

        temp = self.softmax(temp.flatten()/self.temperature)

        maps = temp.reshape(maps.shape)

        return maps.squeeze(0) # (H, W)


def generate_point(density:torch.Tensor, 
                   target_size: tuple
                   ) -> tuple [int, int]:

    """
    Generate a point from a density map

    Args:
        density: A tensor of shape (H, W)
        og_size: The size of the original image (H, W)
    Returns:
        A tensor of shape (2,) containing the coordinates of the point

    """

    H, W = density.shape

    h,w = target_size


    

    density = density.flatten()

    density = density / torch.sum(density)

    cum_density = torch.cumsum(density, dim=0)


    u = torch.rand(1).to("cuda")
    # the cumulative density is sorted in ascending order, so we find the nearest value 
    i = torch.searchsorted(cum_density, u).cpu()
    # we get the coordinates of the point
    x = i % W
    y = i // W


    # the crop must be within the image
    for _ in range(10):

        if (y - h // 2 < 0 or y + h // 2 >= H or x - w // 2 < 0 or x + w // 2 >= W):
            u = torch.rand(1).to("cuda")
            i = torch.searchsorted(cum_density, u).cpu()
            x = i % W
            y = i // W
        else : return x,y # int, int

    # if we can't find a point, we return the center of the image (fallback to center crop)
    return W//2, H//2


In [ ]:
tr = DiNORandomResizedCrop(224, "dinov2_vits14",
                           #scale=(0.1,0.1),
                           temperature=10,
                           kernel_size=5,
                           antialias=True,
                           )

In [ ]:
from PIL import Image
from torchvision import transforms

device = "cuda"
def preprocess_img(path_to_img, resize=False):
    """img = cv2.imread(path_to_img)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = Image.fromarray(img)"""

    img = Image.open(path_to_img)
    if img.mode != "RGB":
        img = img.convert("RGB")
    
    if resize:img = transforms.Resize(500, antialias=True)(img)
    img = transforms.RandomAutocontrast(p=1)(img)

    h, w = img.size
    new_w = w - (w % 14)
    new_h = h - (h % 14)


    img = transforms.ToTensor()(img)

    img = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])(img) 

    img = transforms.Resize((new_w, new_h), antialias=True)(img)

    img = img.unsqueeze(0).to(device)

    return img


In [ ]:
import os
import random as rd

import matplotlib.pyplot as plt

root_imagenet = '/mnt/data/fred/datasets/miniimagenetimages/images/'

img_idx_r = rd.randint(0, len(os.listdir(root_imagenet)) - 1)

#img_idx_r = 2468 #53367 # 32337 #

img_name = os.path.join(
        root_imagenet,
        os.listdir(root_imagenet)[img_idx_r]
    )

#img_name = "/home/someone/Pictures/Screenshots/Screenshot from 2023-08-29 09-33-59.png"
print(img_idx_r)
plt.imshow(Image.open(img_name).convert("RGB"))
plt.axis("off")
plt.show()
img = preprocess_img(img_name, )


In [ ]:
import numpy as np
plt.imshow(Image.open(img_name).convert("RGB"))
to_display = tr.get_map(img).detach().cpu().numpy()
plt.imshow(to_display,alpha=0.8)


plt.colorbar()
plt.show()

In [ ]:
transformed = tr(img)

print(transformed.shape)

# norm
transformed = (transformed - img.min())/(img.max() - img.min())

transformed = transformed.squeeze().permute(1,2,0).cpu().numpy()

plt.imshow(transformed)

In [ ]:
plt.imshow(
    ((transforms.RandomResizedCrop(224, 
                                   #scale=(0.1,0.1),
                                   )(img).squeeze() - img.min())/(img.max() - img.min())).permute(1,2,0).cpu().numpy()
    )

# Test time


In [ ]:
import torchvision
from torchvision import transforms
import torch


train = torchvision.datasets.ImageFolder(
    root="/mnt/data/CUB_200_2011/images/",
    transform= transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomAutocontrast(p=1),
        tr
        ])
)
loader = torch.utils.data.DataLoader(
    train,
    batch_size=2,)

from tqdm import tqdm

In [ ]:
#from dino_transform import DiNORandomResizedCrop

tr = DiNORandomResizedCrop(size=(224,224), 
                           dino_type="dinov2_vits14",
                           #scale=(0.1,0.1),
                           temperature=10,
                           kernel_size=5,
                           antialias=True,
                           )

In [ ]:
%%time
## bs 8
for img, _ in tqdm(loader,total=len(loader)):
    pass
print()

In [ ]:
%%time
## bs 32
for img, _ in tqdm(loader,total=len(loader)):
    pass
print()

In [ ]:
%%time
## bs 32 search on gpu
for img, _ in tqdm(loader,total=len(loader)):
    pass
print()

In [ ]:
%%time
for img, _ in tqdm(loader,total=len(loader)):
    pass

In [ ]:
from config import cfg
from dataset import load_cub_datasets
cfg.use_dino = True
train_loader,val_loader = load_cub_datasets(cfg)

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms
for imgs, _ in train_loader:
    print(imgs.shape)
    for img in imgs:
        #img = img.permute(1,2,0)
        img = (img - img.min())/(img.max() - img.min())
        img = transforms.ToPILImage()(img)
        plt.imshow(img)
        plt.show()
    break